<a href="https://colab.research.google.com/github/Agentoma/mmai5400A_assignment3/blob/main/assignment3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install ipywidgets --upgrade
!pip install jupyter notebook --upgrade
!jupyter nbextension enable --py widgetsnbextension
!pip install "setfit[absa]"
!pip install spacy
!pip install tokenizers>=0.20,<0.21
!pip install transformers -U
!spacy download en_core_web_lg
!spacy download en_core_web_sm


usage: jupyter [-h] [--version] [--config-dir] [--data-dir] [--runtime-dir] [--paths] [--json]
               [--debug]
               [subcommand]

Jupyter: Interactive Computing

positional arguments:
  subcommand     the subcommand to launch

options:
  -h, --help     show this help message and exit
  --version      show the versions of core jupyter packages and exit
  --config-dir   show Jupyter config dir
  --data-dir     show Jupyter data dir
  --runtime-dir  show Jupyter runtime dir
  --paths        show all Jupyter paths. Add --json for machine-readable format.
  --json         output paths as machine-readable json
  --debug        output debug information about paths

Available subcommands: console dejavu events execute kernel kernelspec lab labextension labhub
migrate nbclassic nbconvert notebook run server troubleshoot trust

Jupyter command `jupyter-nbextension` not found.
/bin/bash: line 1: 0.21: No such file or directory
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 587.7

In [2]:
from google.colab import output
output.enable_custom_widget_manager()

Support for third party widgets will remain active for the duration of the session. To disable support:

In [3]:
from google.colab import output
output.disable_custom_widget_manager()

# New Section

In [ ]:
import warnings
from tqdm import TqdmWarning
warnings.filterwarnings('ignore', category=TqdmWarning)

from tqdm.auto import tqdm
import numpy as np
import pandas as pd
from setfit import AbsaModel, AbsaTrainer
from transformers import TrainingArguments, EarlyStoppingCallback, AutoTokenizer
from setfit import AbsaModel, AbsaTrainer, TrainingArguments
from datasets import load_dataset
from transformers import EarlyStoppingCallback
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
from setfit import AbsaModel, AbsaTrainer
from transformers import TrainingArguments

# Load the fine-tuned model

In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
from datasets import load_dataset

# Load a sample dataset
dataset = load_dataset("glue", "mrpc")

# Load tokenizer and model
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
model = AutoModelForSequenceClassification.from_pretrained("bert-base-uncased")

# Tokenize the dataset
def tokenize_function(examples):
    return tokenizer(examples["sentence1"], examples["sentence2"], truncation=True, padding="max_length")

tokenized_datasets = dataset.map(tokenize_function, batched=True)

# Set up training arguments
training_args = TrainingArguments(
    output_dir="./results",
    num_train_epochs=3,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir="./logs",
)

# Initialize Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
)

# Try training
try:
    trainer.train()
except Exception as e:
    print(f"An error occurred: {e}")
    # Print additional debugging information
    print("Dataset info:", tokenized_datasets)
    print("Model config:", model.config)

2024-11-30 22:41:20.083501: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-11-30 22:41:20.215624: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-11-30 22:41:20.266508: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-11-30 22:41:20.433922: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-11-30 22:41:25.599201: W tensorflow/compiler/tf2

Map:   0%|          | 0/408 [00:00<?, ? examples/s]

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
ERROR:wandb.jupyter:Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: adrielgomalley (adrielgomalley-schulich-50). Use `wandb login --relogin` to force relogin


Step,Training Loss


In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("your-model-name")
tokenized_datasets = datasets.map(lambda examples: tokenizer(examples["text"], truncation=True, padding="max_length"), batched=True)

In [ ]:
model = AbsaModel.from_pretrained(
    "sentence-transformers/all-MiniLM-L6-v2",
    spacy_model="en_core_web_sm"
)
model = AbsaModel.from_pretrained(
    "sentence-transformers/all-MiniLM-L6-v2",
    "sentence-transformers/all-mpnet-base-v2",
    spacy_model="en_core_web_sm"
)

In [ ]:
# Third cell: Training setup and execution
# Create sample data - replace with your actual data
train_data = pd.DataFrame({
    'text': ['The food was great', 'Service was poor'],
    'span': ['food', 'service'],
    'polarity': [1, -1]
})

# Convert DataFrame to Dataset
from datasets import Dataset
train_dataset = Dataset.from_pandas(train_data)

# Define training arguments with correct parameters
args = TrainingArguments(
    output_dir="models",
    fp16=True,  # Replace use_amp with fp16
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    num_train_epochs=3,
    evaluation_strategy="steps",
    eval_steps=50,
    save_steps=50,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy"
)

# Initialize trainer
trainer = AbsaTrainer(
    model=model,
    args=args,
    train_dataset=train_dataset,
    eval_dataset=train_dataset  # Replace with your eval data
)

# Train
trainer.train()

# Evaluate
metrics = trainer.evaluate()
print(metrics)

In [ ]:
dataset = load_dataset("tomaarsen/setfit-absa-semeval-restaurants", split="train")


In [ ]:
train_dataset = dataset.select(range(128))
eval_dataset = dataset.select(range(128, 256))

In [ ]:
args = TrainingArguments(
    output_dir="models",
    num_epochs=5,
    use_amp=True,
    batch_size=128,
    eval_strategy="steps",
    eval_steps=50,
    save_steps=50,
    load_best_model_at_end=True,
)


In [ ]:
from setfit import AbsaModel, AbsaTrainer
from transformers import TrainingArguments
from datasets import load_dataset
from setfit import AbsaModel

# Download from the 🤗 Hub
model = AbsaModel.from_pretrained(
    "tomaarsen/setfit-absa-bge-small-en-v1.5-restaurants-aspect",
    "tomaarsen/setfit-absa-bge-small-en-v1.5-restaurants-polarity",
)
# Run inference
preds = model("The food was great, but the venue is just way too busy.")


# Load the model
model = AbsaModel.from_pretrained("sentence-transformers/all-MiniLM-L6-v2")

# Load dataset
dataset = load_dataset("tomaarsen/setfit-absa-semeval-restaurants")
train_dataset = dataset["train"]
eval_dataset = dataset["test"]

# Define training arguments
args = TrainingArguments(
    output_dir="models",
    use_amp=True,
    per_device_train_batch_size=32,  # Adjust batch size as needed
    per_device_eval_batch_size=32,
    num_train_epochs=3,
    evaluation_strategy="steps",
    eval_steps=50,
    save_steps=50,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy"
)

# Initialize trainer
trainer = AbsaTrainer(
    model=model,
    args=args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset
)

# Train the model
trainer.train()

# Evaluate
metrics = trainer.evaluate(eval_dataset)
print(metrics)

# Save to Hub
trainer.push_to_hub("tomaarsen/setfit-absa-restaurants")


In [ ]:


splits = {'train': 'data/train-00000-of-00001.parquet', 'test': 'data/test-00000-of-00001.parquet'}
df = pd.read_parquet("hf://datasets/tomaarsen/setfit-absa-semeval-restaurants/" + splits["train"])
train_dataset = dataset.select(range(128))
eval_dataset = dataset.select(range(128, 256))
args = TrainingArguments(
    output_dir="models",
    num_train_epochs=5,
    use_amp=True,
    per_device_train_batch_size=128,
    evaluation_strategy="steps",
    eval_steps=50,
    save_steps=50,
    load_best_model_at_end=True,
)

In [ ]:
{
 "cells": [
  {
   "cell_type": "code",
   "execution_count": 1,
   "metadata": {},
   "outputs": [
    {
     "name": "stdout",
     "output_type": "stream",
     "text": [
      "Collecting setfit[absa]\n",
      "  Using cached setfit-1.1.0-py3-none-any.whl.metadata (12 kB)\n",
      "Collecting datasets>=2.15.0 (from setfit[absa])\n",
      "  Using cached datasets-3.1.0-py3-none-any.whl.metadata (20 kB)\n",
      "Collecting sentence-transformers>=3 (from sentence-transformers[train]>=3->setfit[absa])\n",
      "  Using cached sentence_transformers-3.3.1-py3-none-any.whl.metadata (10 kB)\n",
      "[notice] A new release of pip is available: 24.2 -> 24.3.1\n",
      "[notice] To update, run: python.exe -m pip install --upgrade pip\n"
     ]
    }
   ],
   "source": [
    "!pip install setfit[absa]",
    "!pip install spacy",
    "!pip install en-core-web-lg==3.7.1",
    "!python -m spacy download en_core_web_lg"
   ]
  }
 ]
}

In [1]:
# Install necessary libraries
!pip install "setfit[absa]" spacy
!spacy download en_core_web_sm

# Ensure compatibility
!pip install tokenizers>=0.20,<0.21
!pip install transformers -U

from setfit import AbsaModel, AbsaTrainer, TrainingArguments
from datasets import load_dataset
from transformers import EarlyStoppingCallback
from setfit import AbsaModel, AbsaTrainer
from transformers import TrainingArguments
from datasets import load_dataset

# Initialize the AbsaModel
model = AbsaModel.from_pretrained(
    "sentence-transformers/all-MiniLM-L6-v2",
    spacy_model="en_core_web_sm",
)

# Load the dataset
dataset = load_dataset("tomaarsen/setfit-absa-semeval-restaurants", split="train")
train_dataset = dataset.select(range(128))
eval_dataset = dataset.select(range(128, 256))

# Set up training arguments
args = TrainingArguments(
    output_dir="models",
    num_train_epochs=5,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    evaluation_strategy="steps",
    eval_steps=50,
    save_steps=50,
    load_best_model_at_end=True,
    fp16=True
)

# Initialize and train the AbsaTrainer
trainer = AbsaTrainer(
    model,
    args=args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=5)],
)
trainer.train()

# Save the trained model
trainer.save_model("absa_model")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.2/75.2 kB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 17.2 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.
     ━━

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

...

test-00000-of-00001.parquet:   0%|          | 0.00/46.6k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/3693 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1134 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


AttributeError: 'TrainingArguments' object has no attribute 'copy'

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive
